In [14]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [22]:
# Load Audio File
wav, sr = librosa.load('pubg_clip.wav')

# Down Sample Audio File
wav_down = librosa.resample(wav, sr, target_sr=16000)

In [25]:
import librosa
import numpy as np

x = np.arange(0, 16000)
frame_len, hop_len = 8000, 400
frames = librosa.util.frame(x, frame_length=frame_len, hop_length=hop_len).T
print(frames)

[[    0     1     2 ...  7997  7998  7999]
 [  400   401   402 ...  8397  8398  8399]
 [  800   801   802 ...  8797  8798  8799]
 ...
 [ 7200  7201  7202 ... 15197 15198 15199]
 [ 7600  7601  7602 ... 15597 15598 15599]
 [ 8000  8001  8002 ... 15997 15998 15999]]


In [ ]:
windowed_frames = np.hanning(frame_len).reshape(-1, 1)*frames
# Print windowed frames
for i, frame in enumerate(windowed_frames):
    print("Win Frame {}: {}".format(i, np.round(frame, 3)))